## Anastasia Ermolova Project: Standardization of sports schools names for Go_protect

🔸Description: The Client is receiving data from different sources, the names of sports schools may be filled out differently; it is necessary to develop a solution to standardize them.

🔸Project goal: to create a solution for matching the names of sports schools.

## Loading Data and Libraries

In [3]:
!pip install -q sentence_transformers
from sentence_transformers import SentenceTransformer, util

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score

In [4]:

from google.colab import drive
drive.mount('/content/drive')


csv_file_path1 = '/content/drive/My Drive/Colab Notebooks/Школы.csv'
schools = pd.read_csv(csv_file_path1)

csv_file_path2 = '/content/drive/My Drive/Colab Notebooks/Примерное написание.csv'
variations = pd.read_csv(csv_file_path2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing and EDA

In [5]:
schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [6]:
variations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB


In [7]:
print("Standard Schools Data:")
print(schools.head(15))

print("\nVariations Data:")
print(variations.head(15))

Standard Schools Data:
    school_id                                     name                region
0           1                                 Авангард    Московская область
1           2                                 Авангард     Ямало-Ненецкий АО
2           3                                  Авиатор  Республика Татарстан
3           4                                   Аврора       Санкт-Петербург
4           5                     Ice Dream / Айс Дрим       Санкт-Петербург
5           6                                  Айсберг       Республика Крым
6           7                                  Айсберг     Рязанская область
7           8                                  Айсберг  Свердловская область
8           9                                Звездочка         Северодвинск 
9          10  Академия синхронного катания на коньках                Москва
10         11                             Академия ФКК   Республика Мордовия
11         12                             Академия ФК

In [8]:

print('unique_school_ids:', schools['school_id'].nunique())


unique_school_ids: 306


It seems that in the schools dataset, the reference names of schools are sorted alphabetically, and 'school_id' is assigned to the school in order, based on the fact that each school in the list is unique. Also in the 'Variations' dataset there is a 'school_id' that clearly does not match the 'school_id' in the 'schools' dataset

In [9]:
variations[variations['school_id'] > 306]

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


Most likely this was a filling error - a human factor, or the name of the school has changed over time

In [ ]:
#variations = variations[variations['school_id'] <= 306]

In [10]:
#Let's combine the data in the 'name', 'region' columns, since the name options in the 'name' column contain both
schools['name_region'] = schools['name'] + ' ' + schools['region']
schools.head()

,school_id,name,region,name_region
0,1,Авангард,Московская область,Авангард Московская область
1,2,Авангард,Ямало-Ненецкий АО,Авангард Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан,Авиатор Республика Татарстан
3,4,Аврора,Санкт-Петербург,Аврора Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург,Ice Dream / Айс Дрим Санкт-Петербург


Let's check which schools are located in the same region in 'schools' dataset to see if there are duplicates in the 'schools' dataset - it looks like there are duplicates of schools 36 and 26 in the Bryansk region, most likely these is not only one case like this, it would be nice to add a decoding of school abbreviation, or schools names are very similar, this is bad for the model, but we will not delete them, since the dataset is already small.

In [11]:
region_value = 'Брянская область'  # Замените на нужное значение region

school_ids = schools[schools['region'] == region_value][['school_id', 'name_region']]

print(school_ids)

     school_id                name_region
25          26    Брянск Брянская область
35          36    ГАУ СК Брянская область
177        178  СШ по ФК Брянская область


Let's check which schools are located in the same region in the 'variations' dataset - it looks like there are many duplicates

Recommendation to the customer: Add a new field to fill in by schools, where a unique identifier will be indicated, for example, TAX number, or you can make an application with auto-filling fields, where you can select a region and there will be a drop-down list with schools names.

In [12]:

school_names_in = variations[variations['name'].str.contains('Брянская область')][['name','school_id']]

print(school_names_in)

                                            name  school_id
335                   СШ по ФК, Брянская область        178
798                     ГАУ СК, Брянская область         36
827  Брянская область, СШ по ФКК ГАУ СК "Брянск"         26
828                Брянская область, СК "Брянск"         26
830                  Брянская область, СК Брянск         26
832           Брянская область, ГАУ "СК "Брянск"         26
833           ГАУ "СК "Брянск", Брянская область         26


In [ ]:
#Let's remove abbreviations and signs that do not carry meaning - I gave up on this idea, abbreviations do make sense, without them we get a lot of duplicates

#to_delete = ['№', 'No', 'ООО', 'ГБУ', 'ГУ', 'АНО', 'ТО', 'ФСО', 'МБУ', 'ДО', 'БУ', 'ГАУ', 'ГБУСО', 'ГБОУ' ]

#schools['name_region'] = schools['name_region'].replace(to_delete, '', regex=True)
#variations['name'] = variations['name'].replace(to_delete, '', regex=True)


In [13]:
#The data contains line separators and capital characters - let's clear the data

def cleaning(text):
    # Replacing line breaks with spaces
    text = re.sub(r"(?:\n|\r)", " ", text)
    # Leave only letters, numbers and spaces
    text = re.sub(r"[^А-Яа-яёЁa-zA-Z0-9\s]+", " ", text)
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    # Convert text to lowercase
    text = text.lower()
    return text

In [14]:
schools[['name', 'region', 'name_region']] = schools[['name', 'region', 'name_region']].applymap(cleaning)
variations['name'] = variations['name'].apply(cleaning)

In [15]:
print("Standard Schools Data:")
print(schools.head())

print("\nVariations Data:")
print(variations.head())

Standard Schools Data:
   school_id                name                region  \
0          1            авангард    московская область   
1          2            авангард     ямало ненецкий ао   
2          3             авиатор  республика татарстан   
3          4              аврора       санкт петербург   
4          5  ice dream айс дрим       санкт петербург   

                          name_region  
0         авангард московская область  
1          авангард ямало ненецкий ао  
2        авиатор республика татарстан  
3              аврора санкт петербург  
4  ice dream айс дрим санкт петербург  

Variations Data:
   school_id                                             name
0       1836                                       ооо триумф
1       1836                                 москва ск триумф
2        610                            сшор надежда губернии
3        610  саратовская область гбусо сшор надежда губернии
4        609                                      сш гвоздик

In [16]:
variations.duplicated().sum()

105

There are a lot of duplicates, if I remove the abbreviations there will be even more of them

In [17]:
#Let's remove duplicates
variations.drop_duplicates()

,school_id,name
0,1836,ооо триумф
1,1836,москва ск триумф
2,610,сшор надежда губернии
3,610,саратовская область гбусо сшор надежда губернии
4,609,сш гвоздика
...,...,...
890,3,республика татарстан сшор фсо авиатор
891,3,сшор фсо авиатор республика татарстан
892,3,республика татарстан мбу до сшор фсо авиатор
893,2,янао сш авангард


In [18]:
variations.rename(columns = {'name':'variation_name'}, inplace=True)

## TfidfVectorizer

In [19]:
data_merged = pd.merge(variations, schools, on='school_id')
print("\nMerged Data:")
print(data_merged.head())


Merged Data:
   school_id                                   variation_name  \
0       1836                                       ооо триумф   
1       1836                                 москва ск триумф   
2        610                            сшор надежда губернии   
3        610  саратовская область гбусо сшор надежда губернии   
4        609                                      сш гвоздика   

                    name                 region  \
0             ооо триумф                 москва   
1             ооо триумф                 москва   
2  сшор надежда губернии    саратовская область   
3  сшор надежда губернии    саратовская область   
4            сш гвоздика  удмуртская республика   

                                 name_region  
0                          ооо триумф москва  
1                          ооо триумф москва  
2  сшор надежда губернии саратовская область  
3  сшор надежда губернии саратовская область  
4          сш гвоздика удмуртская республика  


In [20]:
# Create TF-IDF vectors for the school names and variations
vectorizer = TfidfVectorizer().fit(pd.concat([data_merged['name'], data_merged['variation_name']]))
tfidf_matrix = vectorizer.transform(pd.concat([data_merged['name'], data_merged['variation_name']]))

In [21]:
# Compute cosine similarity between the variations and the standard names
cosine_similarities = cosine_similarity(tfidf_matrix[:len(data_merged)], tfidf_matrix[len(data_merged):])

In [22]:
# Add the most similar school name as a new column
data_merged['predicted_name'] = [data_merged['name'][i] for i in cosine_similarities.argmax(axis=1)]

print("\nData with Predicted School Names:")
print(data_merged[['school_id', 'variation_name', 'name', 'predicted_name']].head())


Data with Predicted School Names:
   school_id                                   variation_name  \
0       1836                                       ооо триумф   
1       1836                                 москва ск триумф   
2        610                            сшор надежда губернии   
3        610  саратовская область гбусо сшор надежда губернии   
4        609                                      сш гвоздика   

                    name         predicted_name  
0             ооо триумф             ооо триумф  
1             ооо триумф             ооо триумф  
2  сшор надежда губернии  сшор надежда губернии  
3  сшор надежда губернии  сшор надежда губернии  
4            сш гвоздика            сш гвоздика  


In [23]:
#we evaluate the model by comparing the predicted school names with the actual names.

accuracy = (data_merged['name'] == data_merged['predicted_name']).mean()

y_true = data_merged['name']
y_pred = data_merged['predicted_name']

f1 = f1_score(y_true, y_pred, average='weighted')
print("F1 Score:", f1)
print(f"Model Accuracy: {accuracy:.2f}")

F1 Score: 0.8409733521814052
Model Accuracy: 0.87


## SBERT

In [24]:
model = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [25]:
corpus = model.encode(schools['name_region'].values.tolist(), convert_to_tensor=True)

In [26]:
queries = model.encode(variations['variation_name'].values.tolist(), convert_to_tensor=True)

In [27]:
results = util.semantic_search(queries, corpus, top_k=1)


In [28]:
len(results)


895

In [29]:
matched_ids = []
for result in results:
    match_id = result[0]['corpus_id']
    matched_ids.append(schools.iloc[match_id]['school_id'])



In [30]:
# Add matched school_id to variations dataframe
variations['matched_school_id'] = matched_ids

In [31]:

# Calculate accuracy and F1 score
variations['school_id'] = variations['school_id'].astype(str)
variations['matched_school_id'] = variations['matched_school_id'].astype(str)

accuracy = accuracy_score(variations['school_id'], variations['matched_school_id'])
f1 = f1_score(variations['school_id'], variations['matched_school_id'], average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.7553072625698324
F1 Score: 0.7669264676991704


When a simple model like TfidfVectorizer shows better results compared to a more complex model like Sentence-BERT (SBERT), it can be due to several reasons:

1. Data

Data Quality: Simple models like TfidfVectorizer can perform well if the data is relatively simple and does not require complex transformations to achieve good results.

Data Quantity: SBERT and other transformer models typically require a large amount of data to achieve better results. If the data volume is small, simple models may perform better.

Data Type: In some tasks where keywords and word frequencies are more important, TfidfVectorizer may be more appropriate. SBERT works better for tasks where understanding context and semantics is crucial.

2. Task

Specific Task: For certain tasks (e.g., keyword-based classification), a simple model may perform better because it does not require complex context understanding.

Task Characteristics: If a task has well-defined patterns that are effectively captured by simple models like TF-IDF, more complex models may be unnecessary.

## Fine-Tuned model

In [32]:
# Let's write a function to add variants of names with typos
def add_typo(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 2)
    return word[:idx] + word[idx + 1] + word[idx] + word[idx + 2:]

In [33]:
# Function for data augmentation
def augment_data(name, region):
    augmented_pairs = set()
    for _ in range(7):  # creation of 7 variations for each name and region
        typo_name = add_typo(name)
        typo_region = add_typo(region)
        augmented_pairs.add((typo_name, typo_region))
    return list(augmented_pairs)

In [34]:
# Creating lists to store augmented names and regions
augmented_names = []
augmented_regions = []
school_ids = []

# Applying augmentation to the 'name' and 'region' columns
for _, row in schools.iterrows():
    name = row['name']
    region = row['region']
    school_id = row['school_id']
    augmented_pairs = augment_data(name, region)
    for aug_name, aug_region in augmented_pairs:
        augmented_names.append(aug_name)
        augmented_regions.append(aug_region)
        school_ids.append(school_id)

In [35]:
# Creating a DataFrame with Augmented Data
augmented_data = pd.DataFrame({
    'school_id': school_ids,
    'name': augmented_names,
    'region': augmented_regions
})

# Combining original data with augmented data
combined_data = pd.concat([schools, augmented_data]).reset_index(drop=True)

# Checking the result
print(combined_data.head(15))

    school_id                                     name                region  \
0           1                                 авангард    московская область   
1           2                                 авангард     ямало ненецкий ао   
2           3                                  авиатор  республика татарстан   
3           4                                   аврора       санкт петербург   
4           5                       ice dream айс дрим       санкт петербург   
5           6                                  айсберг       республика крым   
6           7                                  айсберг     рязанская область   
7           8                                  айсберг  свердловская область   
8           9                                звездочка          северодвинск   
9          10  академия синхронного катания на коньках                москва   
10         11                             академия фкк   республика мордовия   
11         12                           

In [36]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   school_id    2380 non-null   int64 
 1   name         2380 non-null   object
 2   region       2380 non-null   object
 3   name_region  306 non-null    object
dtypes: int64(1), object(3)
memory usage: 74.5+ KB


In [37]:
#Divide into test and train sets
train, test = train_test_split(combined_data, train_size=0.8, test_size=0.2, random_state=42)

In [38]:
train.shape , test.shape

((1904, 4), (476, 4))

In [39]:
model = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
# Encoding names and regions into embeddings

train_embeddings = model.encode(train[['name', 'region']].values.tolist(), convert_to_tensor=True)
test_embeddings = model.encode(test[['name', 'region']].values.tolist(), convert_to_tensor=True)

In [41]:

# Function for matching variations with reference school names
def match_variations(variation_embeddings, school_embeddings, train):
    matched_schools = []
    for var_embedding in variation_embeddings:
        cos_scores = util.pytorch_cos_sim(var_embedding, school_embeddings)[0]
        if np.isnan(cos_scores).any():
            matched_schools.append(('Unknown', 'Unknown'))
        else:
            top_match = np.argmax(cos_scores)
            matched_schools.append((train['name'].iloc[int(top_match)], train['school_id'].iloc[int(top_match)]))
    return matched_schools



In [42]:
# Comparison variations with reference school names
matched_results = match_variations(test_embeddings, train_embeddings, train)

In [43]:
# Create test Dataset with comparison results
test['matched_name'], test['matched_id'] = zip(*matched_results)

In [44]:
# Calculation accuracy and F1-score
test['school_id'] = test['school_id'].astype(str)
test['matched_id'] = test['matched_id'].astype(str)

In [45]:
# Filtering matches with 'Unknown'
test_filtered = test[test['matched_id'] != 'Unknown']

In [46]:
# Check if there is filtered data before calculating metrics
if len(test_filtered) > 0:
    accuracy = accuracy_score(test_filtered['school_id'], test_filtered['matched_id'])
    f1 = f1_score(test_filtered['school_id'], test_filtered['matched_id'], average='weighted')
else:
    accuracy = 0
    f1 = 0

print(f"Accuracy: {accuracy}, F1 Score: {f1}")

Accuracy: 0.8886554621848739, F1 Score: 0.8944761237828466



## Overall Conclusion

I conducted a data analysis which revealed that the raw data contains numerous duplicates that are challenging to remove. This affected both the model building process and the metrics.


Recommendation to the customer: Add a new field to fill in by schools, where a unique identifier will be indicated, for example, TAX number, or you can make an application with auto-filling fields, where you can select a region and there will be a drop-down list with schools names.


Approaches for Matching School Name Variations with Standard Names:


Sentence-BERT (SBERT)
SBERT is used to create sentence embeddings that can be effectively compared using cosine similarity. In your case, LaBSE is a variant of SBERT that performs well with multilingual data.


TF-IDF + Cosine Similarity
This method does not involve deep learning but often yields good results for simple tasks. TF-IDF vectorizes the text, and cosine similarity measures the similarity between vectors.


Fine-Tuned Transformers
If you have enough training data, you can fine-tune transformer models (e.g., BERT, RoBERTa) on your specific task of matching school names. This can improve results compared to using pre-trained models.


Methods and Results
Initially, I used TF-IDF + Cosine Similarity for a basic approach with a small dataset, which produced competitive results without complex models.

F1 Score: 0.8409733521814052

Model Accuracy: 0.87

Then, I used SBERT (LaBSE), which is a good starting option for your task. With decent initial results, I applied data augmentation to increase the training dataset, adding seven variations of school names with typos and errors. This improved the metrics to:

Accuracy: 0.8886554621848739

F1 Score: 0.8944761237828466

With more time, further improvements could be achieved by experimenting with models and augmentation techniques.

However, I would be cautious about trusting these values, knowing that the raw data contains many duplicates.